In [38]:
# 1. Thêm thư viện
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout
from PIL import Image
from keras.models import Model
from keras.applications import VGG16
from keras.preprocessing import image
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [39]:
def preprocess_image(file_path, label):
    # Đọc tệp hình ảnh
    image = tf.io.read_file(file_path)
    
    # Giải mã hình ảnh và xử lý lỗi nếu cần
    try:
        image = tf.image.decode_image(image, channels=3, expand_animations=False)
        image = tf.image.resize(image, [150, 150])
        image = image / 255.0  # Chuẩn hóa pixel (giá trị từ 0 đến 1)
    except tf.errors.InvalidArgumentError:
        # Nếu hình ảnh không thể đọc được, thay thế bằng hình ảnh trắng hoặc hình ảnh gốc
        image = tf.zeros([150, 150, 3])  # Hoặc có thể thay thế bằng hình ảnh trắng
    return image, label

def load_dataset(data_dirs, labels, batch_size):
    images = []
    labels_list = []
    for data_dir, label in zip(data_dirs, labels):
        for filename in os.listdir(data_dir):
            file_path = os.path.join(data_dir, filename)
            if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp')):
                images.append(file_path)
                labels_list.append(label)
    
    # Tạo dataset từ các đường dẫn tệp hình ảnh và nhãn
    dataset = tf.data.Dataset.from_tensor_slices((images, labels_list))
    
    # Áp dụng hàm preprocess_image
    dataset = dataset.map(lambda x, y: preprocess_image(x, y), num_parallel_calls=tf.data.experimental.AUTOTUNE)
    
    # Xáo trộn, batch, và prefetch dataset
    dataset = dataset.shuffle(buffer_size=len(images))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    
    return dataset


In [40]:
# Đường dẫn tới thư mục chứa dữ liệu
data_dir_Lilly = r'flower_images\Lilly'
data_dir_Lotus = r'flower_images\Lotus'
data_dir_Orchid = r'flower_images\Orchid'
data_dir_Sunflower= r'flower_images\Sunflower'
data_dir_Tulip = r'flower_images\Tulip'

In [41]:

def check_and_remove_invalid_images(data_dirs):
    valid_extensions = ('.jpg', '.jpeg', '.png', '.gif', '.bmp')
    for data_dir in data_dirs:
        for filename in os.listdir(data_dir):
            if filename.lower().endswith(valid_extensions):
                file_path = os.path.join(data_dir, filename)
                try:
                    # Đọc và giải mã hình ảnh
                    image = tf.io.read_file(file_path)
                    tf.image.decode_image(image, channels=3)
                except tf.errors.InvalidArgumentError:
                    # In thông báo lỗi và xóa tệp không hợp lệ
                    print(f"Invalid image file: {file_path}. Deleting...")
                    os.remove(file_path)

# Kiểm tra và xóa các tệp hình ảnh không hợp lệ
check_and_remove_invalid_images([data_dir_Lilly, data_dir_Lotus,data_dir_Orchid,data_dir_Sunflower,data_dir_Tulip])

In [42]:

# Gán nhãn cho các lớp
data_dirs = [data_dir_Lilly, data_dir_Lotus,data_dir_Orchid,data_dir_Sunflower,data_dir_Tulip]
labels = [0, 1, 2,3,4]


In [43]:
# Tạo các dataset
dataset = load_dataset(data_dirs, labels, batch_size=32)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
total_size = len(dataset)
train_size = int(0.8 * total_size)
train_dataset = dataset.take(train_size)
validation_dataset = dataset.skip(train_size)

In [44]:
# Xây dựng mô hình
input_shape = (150, 150, 3)
num_classes = 5

In [45]:
model = tf.keras.Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

In [46]:
# Display the model summary
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,828,997 (18.42 MB)

 Trainable params: 4,828,997 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

In [47]:

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # or 'categorical_crossentropy' if using one-hot encoding
              metrics=['accuracy'])

In [48]:
from tensorflow.keras.callbacks import EarlyStopping
# Định nghĩa callback EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',    # Theo dõi giá trị 'val_loss'
    patience=3,            # Số epoch không cải thiện trước khi dừng
    restore_best_weights=True  # Khôi phục trọng số của mô hình tốt nhất
)

In [49]:
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=20,  # Số lượng epoch có thể thay đổi
    verbose=1,  # Hiển thị thông tin huấn luyện
    callbacks=[early_stopping]  # Thêm callback vào danh sách
)

Epoch 1/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 61s 400ms/step - accuracy: 0.3787 - loss: 1.4579 - val_accuracy: 0.5736 - val_loss: 1.0540
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 85s 543ms/step - accuracy: 0.5652 - loss: 1.0990 - val_accuracy: 0.6667 - val_loss: 0.9244
Epoch 3/20
 46/125 ━━━━━━━━━━━━━━━━━━━━ 30s 386ms/step - accuracy: 0.6690 - loss: 0.9328

In [ ]:

def plot_training_history(history):
    # Lấy dữ liệu từ đối tượng history
    history_dict = history.history
    
    # Tạo các biểu đồ cho loss và accuracy
    epochs = range(1, len(history_dict['loss']) + 1)
    
    # Vẽ biểu đồ loss
    plt.figure(figsize=(14, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history_dict['loss'], 'bo-', label='Training loss')
    plt.plot(epochs, history_dict['val_loss'], 'ro-', label='Validation loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    # Vẽ biểu đồ accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, history_dict['accuracy'], 'bo-', label='Training accuracy')
    plt.plot(epochs, history_dict['val_accuracy'], 'ro-', label='Validation accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# Vẽ biểu đồ từ đối tượng history
plot_training_history(history)

In [ ]:
model.save_weights('model_CNN.weights.h5')

In [ ]:
import matplotlib.image as mpimg

# Danh sách ánh xạ các lớp với tên lớp cụ thể
class_names = {0: 'Lilly', 1: 'Lotus', 2: 'Orchid', 3:'Sunflower',4:'Tulip'}

def convert_image_format(image_path, target_format='jpeg'):
    base_name, _ = os.path.splitext(image_path)
    new_image_path = f"{base_name}.{target_format}"
    
    with Image.open(image_path) as img:
        img.save(new_image_path, format=target_format.upper())
    
    return new_image_path

def load_and_preprocess_image(image_path, target_size=(150, 150)):
    if not image_path.lower().endswith(('.jpg', '.jpeg', '.png')):
        image_path = convert_image_format(image_path)
    
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=3, expand_animations=False)
    image = tf.image.resize(image, target_size)
    image = image / 255.0
    image = tf.expand_dims(image, axis=0)
    return image

def predict_image(model, image_path):
    image = load_and_preprocess_image(image_path)
    predictions = model.predict(image)
    predicted_class_index = tf.argmax(predictions[0]).numpy()
    predicted_prob = tf.reduce_max(predictions[0]).numpy()
    return predicted_class_index, predicted_prob

def display_prediction(image_path, model):
    predicted_class_index, predicted_prob = predict_image(model, image_path)
    
    # Ánh xạ chỉ số lớp dự đoán với tên lớp
    predicted_class_name = class_names.get(predicted_class_index, 'Unknown')
    
    print(f"Image path: {image_path}")
    print(f"Predicted class: {predicted_class_name}")
    print(f"Prediction probability: {predicted_prob:.4f}")

    img = mpimg.imread(image_path)
    plt.imshow(img)
    plt.title(f"Predicted class: {predicted_class_name}")
    plt.axis('off')
    plt.show()

# Đường dẫn tới bức ảnh cần kiểm tra
test_image_path = 'R.jpg'  # Thay thế bằng đường dẫn của bạn

# Hiển thị dự đoán
display_prediction(test_image_path, model)
